In [ ]:
import pymongo
from bson.json_util import dumps
uri = "<your_atlas_uri>"
client = pymongo.MongoClient(uri)
mflix = client.sample_mflix
movies = mflix.movies

In [ ]:
sam_raimi_cursor = movies.find( { "directors": "Sam Raimi" } )

In [ ]:
sam_raimi_cursor

In [ ]:
list(sam_raimi_cursor)

In [ ]:
match_stage = {"$match": { "directors": "Sam Raimi" } }
pipeline = [
    match_stage
]
sam_raimi_aggregation = movies.aggregate( pipeline )

In [ ]:
sam_raimi_aggregation

In [ ]:
list(sam_raimi_aggregation)

In [ ]:
sam_raimi_cursor = movies.find(
    { "directors": "Sam Raimi" },
    { "_id": 0, "title": 1, "cast": 1 }
)

print(dumps(sam_raimi_cursor, indent=2))

In [ ]:
match_stage = { "$match": { "directors": "Sam Raimi" } }
project_stage = { "$project": { "_id": 0, "title": 1, "cast": 1 } }

pipeline = [
    match_stage,
    project_stage
]

sam_raimi_aggregation = movies.aggregate( pipeline )

print(dumps(sam_raimi_aggregation, indent=2))

In [ ]:
unwind_stage = { "$unwind": "$directors" }

group_stage = {
    "$group": {
        "_id": {
            "director": "$directors"
        },
        "average_rating": { "$avg": "$imdb.rating" }
    }
}

sort_stage = {
    "$sort": { "average_rating": -1 }
}

# create pipeline from four different stages
pipeline = [
    unwind_stage,
    group_stage,
    sort_stage
]

# aggregate using pipeline
director_ratings = movies.aggregate(pipeline)

# iterate through the resulting cursor
list(director_ratings)